In [1]:
import time
import numpy as np
import pandas as ps
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
from harp_beam import compute_EEPs
from functions import to_dBV, stefcal, stefcal_optimised
from plots import plot2

In [2]:
# set random seed
np.random.seed(12042000)

In [3]:
## Q 2. plot all the 256 EEPs and their average (AEP)
num_dir = 256
theta = np.linspace(-np.pi/2, np.pi/2, num_dir)
phi = np.zeros_like(theta)

# Compute EEPs
v_theta_polY, v_phi_polY, v_theta_polX, v_phi_polX = compute_EEPs(theta.copy()[:, None], phi.copy()[:, None])

# Calculate magnitude of EEPs in dBV
magnitude_EEP_polY = to_dBV(np.abs(v_theta_polY))
magnitude_EEP_polX = to_dBV(np.abs(v_theta_polX))

# Calculate AEPs
AEP_polY = to_dBV(np.sqrt(np.mean(np.abs(v_theta_polY)**2, axis=1)))
AEP_polX = to_dBV(np.sqrt(np.mean(np.abs(v_theta_polX)**2, axis=1)))

# plot EEPs and AEPs
#plot2(theta, v_theta_polY, v_theta_polX, magnitude_EEP_polY, magnitude_EEP_polX, AEP_polY, AEP_polX)

In [4]:
## Q 3. the model matrix, the covariance matrix, the exact gain values and (my) gain estimations are loaded as

filename_vismat = f"data_20feb2024_2330_100MHz.mat"
mat = scipy.io.loadmat(filename_vismat)
R = np.array(mat['R']) # covariance matrix
M_AEP = np.array(mat['M_AEP']) # model matrix using AEP
M_EEPs = np.array(mat['M_EEPs']) # model matrix using all EEPs
g_sol = np.array(mat['g_sol']) # exact gain solution
g_AEP = np.array(mat['g_AEP']) # estimation using M_AEP (using this for question 5 and 6 only if you haven't been able to complete question 3 and 4)
g_EEPs = np.array(mat['g_EEPs']) # estimation using M_EEPs

# StEFCal
M = M_AEP
R = R
max_iteration = 100
threshold = 1e-6

# Algorithm 1 StEFCal
start = time.time()
G = stefcal(M, R, max_iteration, threshold)
end = time.time()
print(f"Time taken for StEFCal: {end - start} seconds")

# Algorithm 2 StEFCal_optimised
start = time.time()
G_opt = stefcal_optimised(M, R, max_iteration, threshold)
end = time.time()
print(f"Time taken for StEFCal_optimised: {end - start} seconds")

# chek diffferenes in G and G_opt
print(f"Difference between G and G_opt: {np.max(np.abs(G - G_opt))}")

/Users/maxtalberg/Documents/Max/Cambridge2324/Lent/SKA/Coursework/harp_beam/functions.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
  G[p, p] = gp  # Update the gain for antenna p in the matrix


Convergence reached after 39 iterations.
Time taken for StEFCal: 1.1860899925231934 seconds


/Users/maxtalberg/Documents/Max/Cambridge2324/Lent/SKA/Coursework/harp_beam/functions.py:57: ComplexWarning: Casting complex values to real discards the imaginary part
  G[p, p] = gp  # Update the gain for antenna p in the matrix


Convergence reached after 27 iterations.
Time taken for StEFCal_optimised: 0.8197376728057861 seconds
Difference between G and G_opt: 2.342878191052833e-05


In [5]:
g_sol

array([[1.44380875e+00+0.00000000e+00j],
       [8.78729641e-01+5.64947996e-01j],
       [1.22877345e+00+9.05358968e-01j],
       [9.25568562e-02+8.66139142e-02j],
       [5.09692804e-01+1.36056049e+00j],
       [7.18555589e-02+7.83245810e-01j],
       [1.06503032e+00+5.98192700e-01j],
       [1.83805464e-03+3.91099811e-04j],
       [1.27397945e-01+1.25862859e+00j],
       [5.57754980e-05+8.30005652e-05j],
       [2.20289924e-01+1.13813385e+00j],
       [6.14892550e-01+7.92198452e-01j],
       [6.16823026e-01+8.27182688e-01j],
       [1.24701791e+00+7.00940751e-01j],
       [7.98133877e-01-3.60527788e-02j],
       [5.29043679e-01+5.93388901e-01j],
       [5.36726726e-01+9.57315560e-01j],
       [1.00501446e+00+1.36747060e+00j],
       [7.66096478e-01+6.43891063e-03j],
       [6.36379066e-01+9.71373212e-02j],
       [1.26406190e-01+2.47166641e-01j],
       [1.38739225e+00+1.14489835e+00j],
       [1.18228617e+00+6.68856936e-01j],
       [1.13025441e+00+4.30553959e-01j],
       [1.684303

In [6]:
G

array([[1.2030785 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.13905793, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.42989764, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.73081394, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.83648703,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.48974226]])

In [12]:
# implement optimised StEFCal algorithm
def stefcal_optimised(M, R, max_iteration=100, threshold=1e-6):
    # Initial gain matrix G
    G = np.eye(len(M)) # Identity matrix

    # Iterative loop
    for i in range(max_iteration):
        # Last iteration of G for comparison
        G_prev = G.copy() 

        for p in range(G.shape[0]):  # Loop over antennas p
            z = np.dot(G, M[:, p])  # Use all rows of M_AEP for antenna p
            gp = np.dot(np.conjugate(R[:, p]).T, z) / np.dot(np.conjugate(z).T, z)  # Calculate new gain for antenna p
            G[p, p] = gp  # Update the gain for antenna p in the matrix

        # Convergence check at every iteration
        if np.linalg.norm(G - G_prev, 'fro') / np.linalg.norm(G, 'fro') < threshold:
            print(f"Convergence reached after {i+1} iterations.")
            break

        # Averaging step only on even iterations
        if i % 2 == 0:
            G = (G + G_prev) / 2

    return G

start = time.time()
G_opt = stefcal_optimised(M, R, max_iteration, threshold)
end = time.time()
print(f"Time taken for StEFCal_optimised: {end - start} seconds")

/var/folders/2g/9t7c77957pl9vb9x2t79vs2c0000gn/T/ipykernel_30977/3038952211.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  G[p, p] = gp  # Update the gain for antenna p in the matrix


Convergence reached after 27 iterations.
Time taken for StEFCal_optimised: 1.2560009956359863 seconds


In [13]:
import numpy as np

max_iteration = 100
threshold = 1e-6
def stefcal_1(M, R, max_iteration=100, threshold=1e-6):
    G = np.eye(len(g_sol))  # Assuming g_sol is the initial guess for the gains

    # Iterative loop
    for i in range(max_iteration):
        G_prev = G.copy()  # Keep a copy of the previous gain matrix for convergence check
        for p in range(G.shape[0]):  # Loop over antennas
            z = np.dot(G, M_AEP[:, p])  # Use all rows of M_AEP for antenna p
            gp = np.dot(np.conjugate(R[:, p]).T, z) / np.dot(np.conjugate(z).T, z)  # Calculate new gain for antenna p
            G[p, p] = gp  # Update the gain for antenna p in the matrix

        # Convergence check every 2 iterations
        if i % 2 == 0:
            delta_G = np.linalg.norm(G - G_prev, 'fro') / np.linalg.norm(G, 'fro')
            if delta_G < threshold:
                print(f"Convergence reached after {i+1} iterations.")
                break
        # Averaging step
        G = (G + G_prev) / 2
    return G

start = time.time()
G = stefcal_1(M_AEP, R, max_iteration, threshold)
end = time.time()
print(f"Time taken for StEFCal: {end - start} seconds")

/var/folders/2g/9t7c77957pl9vb9x2t79vs2c0000gn/T/ipykernel_31510/3186603453.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  G[p, p] = gp  # Update the gain for antenna p in the matrix


Convergence reached after 45 iterations.
Time taken for StEFCal: 1.8787691593170166 seconds
